# Database Normalization: 1NF, 2NF, 3NF, BCNF

**Normalization** is the process of organizing data in a relational database to reduce redundancy and improve data integrity. It involves decomposing tables into smaller, well-structured tables while preserving relationships.

## Why Normalize?

| Problem | Description |
|---------|-------------|
| **Update Anomaly** | Updating data in one place but not others leads to inconsistency |
| **Insert Anomaly** | Cannot insert data without unrelated data being present |
| **Delete Anomaly** | Deleting data unintentionally removes other important information |

---

## Setup: Sample Denormalized Data

Let's start with a completely denormalized table representing student course enrollments with instructor information.

In [ ]:
import pandas as pd

# Denormalized data with multiple issues
denormalized_data = {
    'student_id': [1, 1, 2, 2, 3],
    'student_name': ['Alice', 'Alice', 'Bob', 'Bob', 'Charlie'],
    'student_email': ['alice@uni.edu', 'alice@uni.edu', 'bob@uni.edu', 'bob@uni.edu', 'charlie@uni.edu'],
    'courses': ['Math, Physics', 'Math, Physics', 'Math, Chemistry', 'Math, Chemistry', 'Physics'],
    'course_credits': ['3, 4', '3, 4', '3, 3', '3, 3', '4'],
    'instructor_id': [101, 101, 101, 102, 101],
    'instructor_name': ['Dr. Smith', 'Dr. Smith', 'Dr. Smith', 'Dr. Johnson', 'Dr. Smith'],
    'department': ['Science', 'Science', 'Science', 'Science', 'Science'],
    'dept_building': ['Hall A', 'Hall A', 'Hall A', 'Hall A', 'Hall A']
}

df_denorm = pd.DataFrame(denormalized_data)
print("=== DENORMALIZED TABLE ===")
print("Problems: Repeating groups, redundancy, update/insert/delete anomalies\n")
df_denorm

---

## First Normal Form (1NF)

### Rules for 1NF:
1. **Atomic values**: Each cell contains only a single value (no lists, sets, or repeating groups)
2. **Unique column names**: Each column has a unique name
3. **Unique rows**: Each row is uniquely identifiable (primary key exists)
4. **Single type per column**: All values in a column are of the same data type

### Example Violation:
```
| student_id | courses           | course_credits |
|------------|-------------------|----------------|
| 1          | Math, Physics     | 3, 4           |  ← NOT atomic!
```

### Fix:
Split multi-valued attributes into separate rows.

In [ ]:
# Convert to 1NF: Eliminate repeating groups and ensure atomic values

data_1nf = {
    'student_id': [1, 1, 2, 2, 3],
    'student_name': ['Alice', 'Alice', 'Bob', 'Bob', 'Charlie'],
    'student_email': ['alice@uni.edu', 'alice@uni.edu', 'bob@uni.edu', 'bob@uni.edu', 'charlie@uni.edu'],
    'course_name': ['Math', 'Physics', 'Math', 'Chemistry', 'Physics'],
    'course_credits': [3, 4, 3, 3, 4],
    'instructor_id': [101, 102, 101, 102, 102],
    'instructor_name': ['Dr. Smith', 'Dr. Johnson', 'Dr. Smith', 'Dr. Johnson', 'Dr. Johnson'],
    'department': ['Science', 'Science', 'Science', 'Science', 'Science'],
    'dept_building': ['Hall A', 'Hall A', 'Hall A', 'Hall A', 'Hall A']
}

df_1nf = pd.DataFrame(data_1nf)
print("=== FIRST NORMAL FORM (1NF) ===")
print("✓ Each cell contains atomic (single) values")
print("✓ Primary Key: (student_id, course_name)\n")
df_1nf

---

## Second Normal Form (2NF)

### Rules for 2NF:
1. **Must be in 1NF**
2. **No partial dependencies**: Every non-key attribute must depend on the *entire* primary key, not just part of it

### Identifying Partial Dependencies:

With composite key `(student_id, course_name)`:
- `student_name` depends only on `student_id` → **Partial dependency!**
- `student_email` depends only on `student_id` → **Partial dependency!**
- `course_credits` depends only on `course_name` → **Partial dependency!**

### Fix:
Remove partial dependencies by creating separate tables.

In [ ]:
# Convert to 2NF: Remove partial dependencies

# Table 1: Students (attributes depending on student_id)
students_2nf = pd.DataFrame({
    'student_id': [1, 2, 3],
    'student_name': ['Alice', 'Bob', 'Charlie'],
    'student_email': ['alice@uni.edu', 'bob@uni.edu', 'charlie@uni.edu']
})

# Table 2: Courses (attributes depending on course_name)
courses_2nf = pd.DataFrame({
    'course_name': ['Math', 'Physics', 'Chemistry'],
    'course_credits': [3, 4, 3],
    'instructor_id': [101, 102, 102],
    'instructor_name': ['Dr. Smith', 'Dr. Johnson', 'Dr. Johnson'],
    'department': ['Science', 'Science', 'Science'],
    'dept_building': ['Hall A', 'Hall A', 'Hall A']
})

# Table 3: Enrollments (junction table with full key dependency)
enrollments_2nf = pd.DataFrame({
    'student_id': [1, 1, 2, 2, 3],
    'course_name': ['Math', 'Physics', 'Math', 'Chemistry', 'Physics'],
    'enrollment_date': ['2024-01-15', '2024-01-15', '2024-01-16', '2024-01-16', '2024-01-17']
})

print("=== SECOND NORMAL FORM (2NF) ===")
print("\n📋 STUDENTS TABLE (PK: student_id)")
print(students_2nf.to_string(index=False))
print("\n📋 COURSES TABLE (PK: course_name)")
print(courses_2nf.to_string(index=False))
print("\n📋 ENROLLMENTS TABLE (PK: student_id, course_name)")
print(enrollments_2nf.to_string(index=False))
print("\n✓ No partial dependencies - all non-key attributes depend on full primary key")

---

## Third Normal Form (3NF)

### Rules for 3NF:
1. **Must be in 2NF**
2. **No transitive dependencies**: Non-key attributes must not depend on other non-key attributes

### Identifying Transitive Dependencies:

In the COURSES table:
- `instructor_name` depends on `instructor_id`, not on `course_name` → **Transitive!**
- `dept_building` depends on `department`, not on `course_name` → **Transitive!**

```
course_name → instructor_id → instructor_name (Transitive!)
course_name → department → dept_building (Transitive!)
```

### Fix:
Create separate tables for transitively dependent attributes.

In [ ]:
# Convert to 3NF: Remove transitive dependencies

# Table 1: Students (unchanged from 2NF)
students_3nf = pd.DataFrame({
    'student_id': [1, 2, 3],
    'student_name': ['Alice', 'Bob', 'Charlie'],
    'student_email': ['alice@uni.edu', 'bob@uni.edu', 'charlie@uni.edu']
})

# Table 2: Instructors (extracted from courses)
instructors_3nf = pd.DataFrame({
    'instructor_id': [101, 102],
    'instructor_name': ['Dr. Smith', 'Dr. Johnson'],
    'department': ['Science', 'Science']
})

# Table 3: Departments (extracted to remove transitive dependency)
departments_3nf = pd.DataFrame({
    'department': ['Science', 'Arts', 'Engineering'],
    'dept_building': ['Hall A', 'Hall B', 'Hall C']
})

# Table 4: Courses (now only direct dependencies)
courses_3nf = pd.DataFrame({
    'course_name': ['Math', 'Physics', 'Chemistry'],
    'course_credits': [3, 4, 3],
    'instructor_id': [101, 102, 102]  # FK to Instructors
})

# Table 5: Enrollments (unchanged)
enrollments_3nf = pd.DataFrame({
    'student_id': [1, 1, 2, 2, 3],
    'course_name': ['Math', 'Physics', 'Math', 'Chemistry', 'Physics'],
    'enrollment_date': ['2024-01-15', '2024-01-15', '2024-01-16', '2024-01-16', '2024-01-17']
})

print("=== THIRD NORMAL FORM (3NF) ===")
print("\n📋 STUDENTS (PK: student_id)")
print(students_3nf.to_string(index=False))
print("\n📋 INSTRUCTORS (PK: instructor_id)")
print(instructors_3nf.to_string(index=False))
print("\n📋 DEPARTMENTS (PK: department)")
print(departments_3nf.to_string(index=False))
print("\n📋 COURSES (PK: course_name, FK: instructor_id)")
print(courses_3nf.to_string(index=False))
print("\n📋 ENROLLMENTS (PK: student_id + course_name)")
print(enrollments_3nf.to_string(index=False))
print("\n✓ No transitive dependencies - all attributes depend directly on primary key")

---

## Boyce-Codd Normal Form (BCNF)

### Rules for BCNF:
1. **Must be in 3NF**
2. **Every determinant must be a candidate key**: For every functional dependency X → Y, X must be a superkey

### Difference from 3NF:
- 3NF allows non-key attributes to determine other non-key attributes in certain cases
- BCNF is stricter: *only* candidate keys can be determinants

### Example BCNF Violation:

Consider a course scheduling scenario where:
- Each student takes one course per time slot
- Each instructor teaches one course per time slot
- Each course is taught by only one instructor

```
| student | course  | instructor |
|---------|---------|------------|
| Alice   | Math    | Smith      |
| Bob     | Math    | Smith      |
| Alice   | Physics | Johnson    |
```

Functional Dependencies:
- `(student, course) → instructor` ✓ (candidate key)
- `course → instructor` ✗ (course is NOT a candidate key!)

This violates BCNF because `course` determines `instructor` but `course` alone is not a candidate key.

In [ ]:
# BCNF Violation Example

bcnf_violation = pd.DataFrame({
    'student': ['Alice', 'Bob', 'Charlie', 'Alice'],
    'course': ['Math', 'Math', 'Physics', 'Physics'],
    'instructor': ['Smith', 'Smith', 'Johnson', 'Johnson']
})

print("=== BCNF VIOLATION ===")
print("Candidate Key: (student, course)")
print("Problem: course → instructor (but course is NOT a superkey!)\n")
print(bcnf_violation.to_string(index=False))
print("\n❌ Instructor 'Smith' is repeated for every Math enrollment")
print("❌ If Smith stops teaching Math, we must update multiple rows")

In [ ]:
# Convert to BCNF: Decompose based on the violating FD

# Table 1: Course-Instructor mapping (course → instructor)
course_instructor_bcnf = pd.DataFrame({
    'course': ['Math', 'Physics'],
    'instructor': ['Smith', 'Johnson']
})

# Table 2: Student enrollments (student, course)
student_course_bcnf = pd.DataFrame({
    'student': ['Alice', 'Bob', 'Charlie', 'Alice'],
    'course': ['Math', 'Math', 'Physics', 'Physics']
})

print("=== BOYCE-CODD NORMAL FORM (BCNF) ===")
print("\n📋 COURSE_INSTRUCTOR (PK: course)")
print(course_instructor_bcnf.to_string(index=False))
print("\n📋 STUDENT_COURSE (PK: student, course)")
print(student_course_bcnf.to_string(index=False))
print("\n✓ Every determinant is now a candidate key")
print("✓ course → instructor is valid (course is PK in its table)")

---

## Summary: Normal Forms Comparison

| Normal Form | Requirement | Eliminates |
|-------------|-------------|------------|
| **1NF** | Atomic values, unique rows | Repeating groups |
| **2NF** | 1NF + No partial dependencies | Partial dependencies on composite key |
| **3NF** | 2NF + No transitive dependencies | Non-key → Non-key dependencies |
| **BCNF** | Every determinant is a candidate key | All remaining anomalies from FDs |

### Dependency Chain:
```
UNF → 1NF → 2NF → 3NF → BCNF → 4NF → 5NF
         ↑      ↑      ↑       ↑
      Atomic  Full   Direct  Superkey
      values  key    depend. determin.
```

In [ ]:
# Visual demonstration of data integrity improvement

def simulate_update_anomaly():
    """Show how normalized data prevents update anomalies"""
    
    print("=" * 60)
    print("SCENARIO: Dr. Smith's name changes to Dr. Smith-Williams")
    print("=" * 60)
    
    # Denormalized: Must update multiple rows
    print("\n❌ DENORMALIZED APPROACH:")
    print("   - Must find ALL rows with 'Dr. Smith'")
    print("   - Risk of missing some rows → DATA INCONSISTENCY")
    print("   - 5 UPDATE operations needed\n")
    
    # Normalized: Single update
    print("✅ NORMALIZED APPROACH (3NF):")
    print("   - Update ONLY the Instructors table")
    print("   - 1 UPDATE operation needed")
    print("   - Data consistency guaranteed\n")
    
    # Demonstrate
    instructors_updated = instructors_3nf.copy()
    instructors_updated.loc[instructors_updated['instructor_id'] == 101, 'instructor_name'] = 'Dr. Smith-Williams'
    
    print("📋 INSTRUCTORS (after single UPDATE):")
    print(instructors_updated.to_string(index=False))

simulate_update_anomaly()

---

## Trade-offs: Normalization vs Denormalization

### When to Normalize (OLTP Systems)

| Advantage | Description |
|-----------|-------------|
| **Data Integrity** | Single source of truth, no anomalies |
| **Storage Efficiency** | No redundant data |
| **Write Performance** | Updates affect fewer rows |
| **Flexibility** | Easier schema evolution |

### When to Denormalize (OLAP/Analytics)

| Advantage | Description |
|-----------|-------------|
| **Read Performance** | Fewer JOINs needed |
| **Query Simplicity** | Flatter data structures |
| **Aggregations** | Pre-computed values |
| **Caching** | Self-contained records |

### Real-World Strategies

```
┌─────────────────┐     ETL      ┌─────────────────┐
│   OLTP System   │ ──────────→  │   Data Warehouse│
│  (Normalized)   │              │ (Denormalized)  │
│   3NF / BCNF    │              │   Star Schema   │
└─────────────────┘              └─────────────────┘
   Writes/Updates                   Analytics/Reports
```

In [ ]:
# Performance comparison: Normalized vs Denormalized queries

print("=== QUERY COMPLEXITY COMPARISON ===")
print("\n📊 Query: Get student names with their courses and instructors\n")

# Denormalized query (simple but redundant data)
print("DENORMALIZED (1 table):")
print("""
SELECT student_name, course_name, instructor_name
FROM student_course_all;
""")
print("→ Simple query, but table has redundant data\n")

# Normalized query (requires JOINs)
print("NORMALIZED (3NF - 4 tables):")
print("""
SELECT s.student_name, c.course_name, i.instructor_name
FROM enrollments e
JOIN students s ON e.student_id = s.student_id
JOIN courses c ON e.course_name = c.course_name
JOIN instructors i ON c.instructor_id = i.instructor_id;
""")
print("→ More complex query, but data is consistent and non-redundant")

# Demonstrate JOIN result
print("\n📋 JOINED RESULT (from normalized tables):")
result = enrollments_3nf.merge(students_3nf, on='student_id')
result = result.merge(courses_3nf, on='course_name')
result = result.merge(instructors_3nf, on='instructor_id')
print(result[['student_name', 'course_name', 'instructor_name']].to_string(index=False))

---

## 📌 Key Takeaways

### The Normalization Journey

1. **1NF**: Eliminate repeating groups → Each cell has ONE atomic value
2. **2NF**: Eliminate partial dependencies → Non-key attributes depend on the WHOLE key
3. **3NF**: Eliminate transitive dependencies → Non-key attributes depend ONLY on the key
4. **BCNF**: Every determinant is a candidate key → Strictest form for functional dependencies

### Practical Guidelines

| Situation | Recommendation |
|-----------|----------------|
| Transactional systems (OLTP) | Normalize to 3NF/BCNF |
| Analytical systems (OLAP) | Denormalize (Star/Snowflake schema) |
| High-read, low-write | Consider strategic denormalization |
| Data warehousing | Use dimensional modeling |
| Microservices | Denormalize per service boundary |

### Remember

> *"Normalize until it hurts, denormalize until it works."*

- Start with a normalized design (3NF is usually sufficient)
- Denormalize selectively based on measured performance needs
- Document any intentional denormalization decisions
- Consider using materialized views for read-heavy denormalization